In [1]:
PROJECT_NAME = 'ds-training-380514'
LOCATION = "us-central1"
MODEL_NAME = "lending_club_xgb"
TARGET_COLUMN = 'is_bad'

In [2]:
from google.cloud import aiplatform

In [3]:
def create_endpoint(
    project: str,
    display_name: str,
    location: str,
):
    """Create an Vertex AI Model Endpoint in the given project and location"""
    
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint

In [4]:
def deploy_model(
    project: str,
    location: str,
    model_name: str,
    endpoint_name: str
):
    """
    model_name: A fully-qualified model resource name or model ID.
    endpoint_name: A fully-qualified endpoint resource name or endpoint ID.
    """

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_name=model_name)
    endpoint = aiplatform.Endpoint(endpoint_name=endpoint_name)

    model.deploy(
        endpoint=endpoint,
        machine_type="e2-standard-4"
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model

In [5]:
from typing import List, Dict

def predict_tabular_classification(
    project: str,
    location: str,
    endpoint_name: str,
    instances: List[Dict],
):
    """
    Args
        project: Your project ID or project number.
        location: Region where Endpoint is located. For example, 'us-central1'.
        endpoint_name: A fully qualified endpoint name or endpoint ID. Example: "projects/123/locations/us-central1/endpoints/456" or
               "456" when project and location are initialized or passed.
        instances: A list of one or more instances (examples) to return a prediction for.
    """
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint_name)

    response = endpoint.predict(instances=instances)

    for prediction_ in response.predictions:
        print(prediction_)

In [6]:
# Note that you don't have to create an endpoint every time you run this notebook
create_endpoint(PROJECT_NAME, f'{MODEL_NAME}_endpoint', LOCATION)

Creating Endpoint
Create Endpoint backing LRO: projects/354621994428/locations/us-central1/endpoints/8069267457736441856/operations/1240176342205464576
Endpoint created. Resource name: projects/354621994428/locations/us-central1/endpoints/8069267457736441856
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/354621994428/locations/us-central1/endpoints/8069267457736441856')
lending_club_xgb_endpoint
projects/354621994428/locations/us-central1/endpoints/8069267457736441856


resource name: projects/354621994428/locations/us-central1/endpoints/8069267457736441856

In [7]:
MODEL_ID = "projects/354621994428/locations/us-central1/models/8670847652688035840"

In [9]:
deploy_model(
    PROJECT_NAME,
    LOCATION,
    MODEL_ID,
    "projects/354621994428/locations/us-central1/endpoints/4923503118018150400")

Deploying model to Endpoint : projects/354621994428/locations/us-central1/endpoints/4923503118018150400
Deploy Endpoint model backing LRO: projects/354621994428/locations/us-central1/endpoints/4923503118018150400/operations/6320236721879384064
Endpoint model deployed. Resource name: projects/354621994428/locations/us-central1/endpoints/4923503118018150400
ml_engine_us-central1_census_income_student_1_v1
projects/354621994428/locations/us-central1/models/8670847652688035840


resource name: projects/354621994428/locations/us-central1/models/8670847652688035840

In [15]:
import pandas as pd
import json
df_inference = pd.read_csv("gs://ds-training-ay3/lending_club/inference/inference_data.csv")
df_inference.head()

,Id,emp_length,annual_inc,zip_code,debt_to_income,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,...,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,policy_code_PC1,policy_code_PC2,policy_code_PC3,policy_code_PC4,policy_code_PC5
0,42,4,250000.0,88,0.98,0,2,120.0,0.0,6,...,0,0,0,0,0,0,0,1,0,0
1,71,1,48000.0,100,2.60,0,0,120.0,0.0,4,...,0,0,0,0,0,0,0,1,0,0
2,91,1,51400.0,223,1.77,0,2,120.0,0.0,7,...,0,0,0,0,0,0,0,1,0,0
3,102,3,61000.0,752,0.98,0,4,120.0,0.0,8,...,0,0,0,0,0,0,1,0,0,0
4,104,2,27600.0,310,21.83,0,0,120.0,0.0,5,...,0,0,0,0,0,1,0,0,0,0


In [16]:
inference_sample = df_inference.iloc[:10,:].reset_index(drop=True)
inference_sample.set_index(["Id"], inplace=True)

In [17]:
for index, row in inference_sample.iterrows():
    instance = json.loads(row.astype(str).to_json())
    inference_results = predict_tabular_classification(PROJECT_NAME, LOCATION, 'projects/354621994428/locations/us-central1/endpoints/4923503118018150400', [instance])


FailedPrecondition: 400 "Prediction failed: Could not initialize DMatrix from inputs: Input numpy.ndarray must be 2 dimensional"
